<a href="https://colab.research.google.com/github/rahulshirke7951/NSE-Indices/blob/main/NSEIndices.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# Unmount safely (does NOT delete anything)

from google.colab import drive
drive.flush_and_unmount()


Drive not mounted, so nothing to flush and unmount.


In [ ]:
# Remove ONLY the local mount folder

!rm -rf /content/drive

In [ ]:
#Recreate it cleanly by mounting again

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
# === DATA CELL ===
# Run this first. It fetches data and writes raw_data.xlsx (no formatting).
!pip install --quiet yfinance pandas openpyxl requests

import os, json
import pandas as pd
import yfinance as yf
from datetime import date

# Paths / config
BASE = "/content/drive/MyDrive/NSE.Indices"
# JSON_CFG = os.path.join(BASE, "Json", "config_nse.json")

JSON_CFG = os.path.join(BASE, "Json", "nse_broad.json")

RAW_XL = os.path.join(BASE, "raw_data.xlsx")          # raw (unformatted) output
CACHE_DIR = os.path.join(BASE, "cache")
os.makedirs(os.path.join(BASE, "Json"), exist_ok=True)
os.makedirs(CACHE_DIR, exist_ok=True)

# Load config
if not os.path.exists(JSON_CFG):
    raise FileNotFoundError(f"Config missing at {JSON_CFG}")
with open(JSON_CFG, "r") as f:
    cfg = json.load(f)

INDICES = cfg.get("indices", {})
START_DATE = cfg.get("start_date")
END_DATE = cfg.get("end_date")
today = date.today()
if START_DATE and END_DATE:
    start_date = pd.to_datetime(START_DATE).date()
    end_date = pd.to_datetime(END_DATE).date()
else:
    start_date = today.replace(day=1)
    end_date = today

print("Date range:", start_date, "→", end_date)
print("Indices:", list(INDICES.keys()))

# Fetch close prices (yfinance), keep ascending index (oldest -> newest)
data_dict = {}
for name, symbol in INDICES.items():
    try:
        df = yf.Ticker(symbol).history(start=start_date,
                                       end=(pd.to_datetime(end_date) + pd.Timedelta(days=1)).date())
        if df is not None and not df.empty:
            data_dict[name] = df["Close"].copy()
            # cache csv for quick manual inspection (optional)
            csvp = os.path.join(CACHE_DIR, f"{symbol.replace('^','caret_')}.csv")
            try:
                df.to_csv(csvp)
            except Exception:
                pass
            print(f"Fetched: {name} ({symbol}) rows={len(df)}")
        else:
            print(f"No data: {name} ({symbol})")
    except Exception as e:
        print(f"Error fetching {name} ({symbol}): {e}")

if not data_dict:
    raise RuntimeError("No data fetched for any index. Fix config symbols.")

# Build df_close (ascending)
df_close = pd.DataFrame(data_dict)
df_close.index = pd.to_datetime(df_close.index).tz_localize(None)
df_close = df_close.sort_index(ascending=True)
df_close = df_close.dropna(axis=1, how="all").ffill().bfill()

# MTD % (from first available day)
df_pct_mtd = (df_close.div(df_close.iloc[0]) - 1) * 100
df_pct_mtd = df_pct_mtd.round(2)

# Day-over-day % (DoD)
df_pct_dod = df_close.pct_change() * 100
df_pct_dod = df_pct_dod.round(2)

# Summary (latest MTD)
mtd_series = df_pct_mtd.iloc[-1].copy()
summary = pd.DataFrame({"MTD % Change": mtd_series}).sort_values(by="MTD % Change", ascending=False)

# Daily Movers (top3 based on DoD)
daily_summary = []
for dt in df_pct_dod.sort_index(ascending=False).index:
    row = df_pct_dod.loc[dt].dropna()
    if not row.empty:
        top_g = row.nlargest(3)
        top_l = row.nsmallest(3)
        daily_summary.append({
            "Date": dt.strftime("%d-%b-%y"),
            "Top 3 Gainers": ", ".join(top_g.index),
            "Top 3 Losers": ", ".join(top_l.index)
        })
daily_summary_df = pd.DataFrame(daily_summary)

# Streaks
streaks = {}
for col in df_close.columns:
    diffs = df_close[col].diff().fillna(0)
    # longest win streak
    max_win = 0; cur = 0
    for v in (diffs > 0):
        if v:
            cur += 1
            if cur > max_win: max_win = cur
        else:
            cur = 0
    # longest lose streak
    max_loss = 0; cur = 0
    for v in (diffs < 0):
        if v:
            cur += 1
            if cur > max_loss: max_loss = cur
        else:
            cur = 0
    streaks[col] = {"Longest Win Streak": int(max_win), "Longest Lose Streak": int(max_loss)}
streaks_df = pd.DataFrame.from_dict(streaks, orient="index").reset_index().rename(columns={"index":"Index"})

# Market overview numbers (saved for convenience)
avg_change = summary["MTD % Change"].mean()
gain_cnt = int((summary["MTD % Change"] > 0).sum())
loss_cnt = int((summary["MTD % Change"] < 0).sum())
best = summary.index[0] if len(summary) else ""
worst = summary.index[-1] if len(summary) else ""
mood = "Bullish" if avg_change > 0 else ("Bearish" if avg_change < 0 else "Neutral")

market_overview = pd.DataFrame({
    "Key Insight": [
        "Date Range","Average Market Change (%)","Total Gainers","Total Losers",
        "Top Performer","Bottom Performer","Market Mood"
    ],
    "Value": [
        f"{start_date.strftime('%d-%b-%Y')} → {end_date.strftime('%d-%b-%Y')}",
        f"{round(avg_change,2)}%","{g}".format(g=gain_cnt), "{l}".format(l=loss_cnt), best, worst, mood
    ]
})

# Save raw (unformatted) Excel
with pd.ExcelWriter(RAW_XL, engine="openpyxl") as w:
    # Write raw data; keep indexes in readable formats
    df_close.sort_index(ascending=False).to_excel(w, sheet_name="Index Close")
    df_pct_mtd.sort_index(ascending=False).to_excel(w, sheet_name="MTD %")
    df_pct_dod.sort_index(ascending=False).to_excel(w, sheet_name="Day over Day %")
    summary.to_excel(w, sheet_name="Summary")
    daily_summary_df.to_excel(w, sheet_name="Daily Movers", index=False)
    streaks_df.to_excel(w, sheet_name="Streaks", index=False)
    market_overview.to_excel(w, sheet_name="Market Overview", index=False)

print("✅ Raw data saved to:", RAW_XL)
print("You can now run the formatting cell (separate) to style this workbook.")


Date range: 2025-09-01 → 2025-11-30
Indices: ['Nifty 50', 'Nifty Next 50', 'Nifty 100', 'Nifty 200', 'Nifty Microcap 250', 'Nifty Smallcap 50', 'Nifty Midcap 50', 'Nifty Smallcap 100', 'Nifty Midcap 100', 'Nifty Midcap 150', 'Nifty MidSmallcap 400', 'Nifty Smallcap 250', 'Nifty Midcap Select', 'Nifty500 LargeMidSmall Equal-Cap Weighted', 'Nifty500 Multicap 50:25:25', 'Nifty LargeMidcap 250', 'Nifty Total Market', 'Nifty 500']
Fetched: Nifty 50 (^NSEI) rows=62
Fetched: Nifty Next 50 (^NSMIDCP) rows=62
Fetched: Nifty 100 (^CNX100) rows=62
Fetched: Nifty 200 (^CNX200) rows=62
Fetched: Nifty Microcap 250 (NIFTY_MICROCAP250.NS) rows=1
Fetched: Nifty Smallcap 50 (NIFTYSMLCAP50.NS) rows=1
Fetched: Nifty Midcap 50 (^NSEMDCP50) rows=62
Fetched: Nifty Smallcap 100 (^CNXSC) rows=1
Fetched: Nifty Midcap 100 (^CRSMID) rows=62
Fetched: Nifty Midcap 150 (NIFTYMIDCAP150.NS) rows=62
Fetched: Nifty MidSmallcap 400 (NIFTYMIDSML400.NS) rows=1
Fetched: Nifty Smallcap 250 (NIFTYSMLCAP250.NS) rows=1
Fetched:

In [ ]:
# === FORMATTING CELL ===
# Run this after DATA CELL. It reads raw_data.xlsx and writes formatted_dashboard.xlsx
!pip install --quiet openpyxl pandas

import os
import pandas as pd
from openpyxl import load_workbook
from openpyxl.styles import Font, Alignment, PatternFill, Border, Side
from openpyxl.formatting.rule import ColorScaleRule
from openpyxl.chart import BarChart, Reference
from openpyxl.chart.label import DataLabelList

BASE = "/content/drive/MyDrive/NSE.Indices"
RAW_XL = os.path.join(BASE, "raw_data.xlsx")
OUT_XL = os.path.join(BASE, "formatted_dashboard.xlsx")

if not os.path.exists(RAW_XL):
    raise FileNotFoundError(f"Raw file not found at {RAW_XL}. Run DATA CELL first.")

# load workbook
wb = load_workbook(RAW_XL)
border = Border(bottom=Side(border_style="thin", color="000000"))

def set_header_style(ws, header_row=1):
    try:
        for c in ws[header_row]:
            c.font = Font(bold=True)
            c.alignment = Alignment(horizontal="center", vertical="center")
            c.fill = PatternFill(start_color="DCE6F1", end_color="DCE6F1", fill_type="solid")
            c.border = border
    except Exception:
        pass

def set_col_widths(ws, widths):
    # widths: dict {col_index_or_name: width} or (first_col, other_col) tuple
    try:
        if isinstance(widths, dict):
            for k,w in widths.items():
                # if k is index (int) use get_column_letter, else assume header name
                try:
                    from openpyxl.utils import get_column_letter
                    if isinstance(k, int):
                        letter = get_column_letter(k)
                    else:
                        # find column index by header name (row1)
                        for cell in ws[1]:
                            if cell.value == k:
                                letter = ws.cell(row=1, column=cell.column).column_letter
                                break
                        else:
                            continue
                    ws.column_dimensions[letter].width = w
                except Exception:
                    pass
        else:
            # widths as tuple (first_col, other_col)
            first, other = widths
            try:
                letter = ws.cell(row=1, column=1).column_letter
                ws.column_dimensions[letter].width = first
            except Exception:
                pass
            for i in range(2, ws.max_column+1):
                try:
                    l = ws.cell(row=1, column=i).column_letter
                    ws.column_dimensions[l].width = other
                except Exception:
                    pass
    except Exception:
        pass

# Apply basic formatting per sheet
for name in wb.sheetnames:
    ws = wb[name]
    set_header_style(ws, header_row=1)

# Specific widths / tweaks
set_col_widths(wb["Index Close"], (16, 14))
set_col_widths(wb["MTD %"], (16, 12))
set_col_widths(wb["Day over Day %"], (16, 12))
set_col_widths(wb["Summary"], (22, 16))
set_col_widths(wb["Daily Movers"], (16, 30))
set_col_widths(wb["Streaks"], (20, 14))
set_col_widths(wb["Market Overview"], (28, 28))

# MTD %: apply heatmap (conditional formatting) — numeric values (not percent-format)
try:
    ws = wb["MTD %"]
    lc, lr = ws.max_column, ws.max_row
    # header is row1 -> data from row2. Use B2..lastcol,lastrow
    rng = f"B2:{chr(65+lc-1)}{lr}" if lc >= 2 else "B2:B2"
    rule = ColorScaleRule(start_type="num", start_value=-2, start_color="F8696B",
                          mid_type="num", mid_value=0,  mid_color="FFEB84",
                          end_type="num", end_value=2,  end_color="63BE7B")
    ws.conditional_formatting.add(rng, rule)
except Exception:
    pass

# Keep numeric format for MTD% and DoD (do NOT change to Excel % format)
# Streaks: ensure integers (set number format)
try:
    ws = wb["Streaks"]
    for colcell in ws[1]:
        header = colcell.value
        if header in ("Longest Win Streak", "Longest Lose Streak"):
            col = colcell.column_letter
            for cell in ws[col]:
                if cell.row > 1:
                    cell.number_format = '0'
except Exception:
    pass

# Market Overview: make sure first column bold and widths OK
try:
    ws = wb["Market Overview"]
    for cell in ws["A"]:
        cell.font = Font(bold=True)  # keys bold
    ws.column_dimensions[ws.cell(row=1,column=1).column_letter].width = 28
    ws.column_dimensions[ws.cell(row=1,column=2).column_letter].width = 36
except Exception:
    pass

# Save formatted workbook
wb.save(OUT_XL)
print("✅ Formatted workbook saved to:", OUT_XL)
print("You can now open formatted_dashboard.xlsx. To tweak styles, edit this FORMATTING CELL only.")


✅ Formatted workbook saved to: /content/drive/MyDrive/NSE.Indices/formatted_dashboard.xlsx
You can now open formatted_dashboard.xlsx. To tweak styles, edit this FORMATTING CELL only.


In [ ]:
# use this for quick test

import yfinance as yf
df = yf.Ticker("0P0001IUFY.BO").history(start="2025-01-01", end="2025-11-30")
print(df.head(), df.tail(), len(df))


                                Open       High        Low      Close  Volume  \
Date                                                                            
2025-01-01 00:00:00+05:30  23.480801  23.480801  23.480801  23.480801       0   
2025-01-02 00:00:00+05:30  23.774099  23.774099  23.774099  23.774099       0   
2025-01-03 00:00:00+05:30  23.804001  23.804001  23.804001  23.804001       0   
2025-01-06 00:00:00+05:30  23.048100  23.048100  23.048100  23.048100       0   
2025-01-07 00:00:00+05:30  23.027599  23.027599  23.027599  23.027599       0   

                           Dividends  Stock Splits  Capital Gains  
Date                                                               
2025-01-01 00:00:00+05:30        0.0           0.0            0.0  
2025-01-02 00:00:00+05:30        0.0           0.0            0.0  
2025-01-03 00:00:00+05:30        0.0           0.0            0.0  
2025-01-06 00:00:00+05:30        0.0           0.0            0.0  
2025-01-07 00:00:00+05:3

In [7]:
# added for test

import yfinance as yf
df = yf.Ticker("^NSEI").history(start="2025-01-01", end="2025-11-30")
print(df.head(), df.tail(), len(df))


                                   Open          High           Low  \
Date                                                                  
2025-01-01 00:00:00+05:30  23637.650391  23822.800781  23562.800781   
2025-01-02 00:00:00+05:30  23783.000000  24226.699219  23751.550781   
2025-01-03 00:00:00+05:30  24196.400391  24196.449219  23976.000000   
2025-01-06 00:00:00+05:30  24045.800781  24089.949219  23551.900391   
2025-01-07 00:00:00+05:30  23679.900391  23795.199219  23637.800781   

                                  Close  Volume  Dividends  Stock Splits  
Date                                                                      
2025-01-01 00:00:00+05:30  23742.900391  154900        0.0           0.0  
2025-01-02 00:00:00+05:30  24188.650391  283200        0.0           0.0  
2025-01-03 00:00:00+05:30  24004.750000  312300        0.0           0.0  
2025-01-06 00:00:00+05:30  23616.050781  278100        0.0           0.0  
2025-01-07 00:00:00+05:30  23707.900391  262300     